# DSA - inner collisions

Find two (meaningful*) messages m_1 and m_2, for which DSA public parameters (p, q, g) exist such that both messages have the same signature (r, s) regardless of the private key value. The signature of m_1 is thus at the same time a valid signature of m_2 and vice versa. Find the parameters (p, q, g) and show the collision of both signatures for a randomly chosen private key.